In [7]:
# prompt: pip install deeplake langchain tensorflow_text google-cloud-aiplatform streamlit

!pip install -q deeplake langchain tensorflow_text google-cloud-aiplatform streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.8/583.8 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7

In [15]:
# prompt: loggin in google with collab and load drive

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
import vertexai
import os
from langchain.llms import VertexAI

PROJECT_ID = 'zeta-yen-319702'
REGION = 'us-central1'
BUCKET = 'gs://zeta-yen-319702-temp/'


vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET
)

# https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#:~:text=PaLM%202%20for%20Text%20(%20text%2Dunicorn%20),with%20complex%20natural%20language%20tasks.


llm = VertexAI(
    #model_name="text-bison@001",
    model_name="text-bison",
    max_output_tokens=256,
    temperature=0.8,
    top_p=0.8,
    top_k=5,
    verbose=False,
)

llm(prompt='Hi how are you?')

" I'm doing great, thanks for asking! How are you?"

In [9]:
!wget 'https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt'  -O alice_in_wonderland.txt
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt" -O Sorcerer_stone.txt

--2024-01-02 22:29:46--  https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148574 (145K) [text/plain]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 145.09K  --.-KB/s    in 0.003s  

2024-01-02 22:29:46 (41.6 MB/s) - ‘alice_in_wonderland.txt’ saved [148574/148574]

--2024-01-02 22:29:46--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserconten

In [10]:
from langchain.embeddings import TensorflowHubEmbeddings
from langchain.vectorstores import DeepLake
url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed_model = TensorflowHubEmbeddings(model_url=url)

db = DeepLake(dataset_path='./deeplk', \
                       embedding=embed_model)

def load_doc(filename,metatag):
  from langchain.document_loaders import TextLoader
  from langchain.text_splitter import CharacterTextSplitter

  loader = TextLoader(filename)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  docs = text_splitter.split_documents(documents)
  for d in docs:
    for key, value in metatag.items():
      d.metadata[key] = value
  return docs


alice_docs = load_doc('alice_in_wonderland.txt' ,{"author": "Lewis", "book": "Alice"})
stone_docs = load_doc('Sorcerer_stone.txt' ,{"author": "Rowling", "book": "Stone"})

lista_docs = db.add_documents(alice_docs)
lists_docs = db.add_documents(stone_docs)

query = "What was the Mad Hatter's riddle about raven and writing desks?"
docs = db.similarity_search(query)
print(docs[0].page_content)

db.similarity_search(
    "What color was the Sorcerer stone?",
    filter={"metadata": {"author": "Rowling"}},
)




Creating 178 embeddings in 1 batches of size 178:: 100%|██████████| 1/1 [00:07<00:00,  7.93s/it]


Dataset(path='./deeplk', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (178, 1)     str     None   
 metadata     json      (178, 1)     str     None   
 embedding  embedding  (178, 512)  float32   None   
    id        text      (178, 1)     str     None   


Creating 512 embeddings in 2 batches of size 500:: 100%|██████████| 2/2 [00:15<00:00,  7.99s/it]


Dataset(path='./deeplk', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (690, 1)     str     None   
 metadata     json      (690, 1)     str     None   
 embedding  embedding  (690, 512)  float32   None   
    id        text      (690, 1)     str     None   
`I'm a poor man, your Majesty,' the Hatter began, in a
trembling voice, `--and I hadn't begun my tea--not above a week
or so--and what with the bread-and-butter getting so thin--and
the twinkling of the tea--'

  `The twinkling of the what?' said the King.

  `It began with the tea,' the Hatter replied.

  `Of course twinkling begins with a T!' said the King sharply.
`Do you take me for a dunce?  Go on!'

  `I'm a poor man,' the Hatter went on, `and most things
twinkled after that--only the March Hare said--'

  `I didn't!' the March Hare interrupted in a great hurry.

  `You did!' said the Hatter.

100%|██████████| 690/690 [00:00<00:00, 2420.08it/s]


[Document(page_content='She pushed the book toward them, and Harry and Ron read: The ancient\nstudy of alchemy is concerned with making the Sorcerer\'s Stone, a\nlegendary substance with astonishing powers. The stone will transform\nany metal into pure gold. It also produces the Elixir of Life, which\nwill make the drinker immortal.\n\nThere have been many reports of the Sorcerer\'s Stone over the centuries,\nbut the only Stone currently in existence belongs to Mr. Nicolas Flamel,\nthe noted alchemist and opera lover. Mr. Flamel, who celebrated his six\nhundred and sixty-fifth birthday last year, enjoys a quiet life in Devon\nwith his wife, Perenelle (six hundred and fifty-eight).\n\n"See?" said Hermione, when Harry and Ron had finished. "The dog must be\nguarding Flamel\'s Sorcerer\'s Stone! I bet he asked Dumbledore to keep it\nsafe for him, because they\'re friends and he knew someone was after it,\nthat\'s why he wanted the Stone moved out of Gringotts!"', metadata={'source': 'Sorc

In [ ]:
retriever=db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 20

retriever.search_kwargs['reduce_k_below_max_tokens'] = True
retriever.search_kwargs['max_tokens_limit'] = 3000
retriever.search_kwargs['filter'] = {"metadata": {"author": "Rowling"}}

In [19]:
# prompt: generate all imports for below code from operator import itemgetter  from langchain_core.runnables import RunnableParallel  template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum and keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. {context} Chat History: {chat_history} Question: {question} Helpful Answer:""" rag_prompt_custom = PromptTemplate.from_template(template)  def format_docs(docs):     return "\n\n".join(doc.page_content for doc in docs)  def format_history(chat_history):     return "\n\n".join(f'{chat.type} : {chat.content}\n' for chat in chat_history)    rag_chain_from_docs = (     {         "context": lambda input: format_docs(input["documents"]),         "question": itemgetter("question"),         "chat_history": lambda input: format_history ([             HumanMessage(content="What does LLM stand for?"),             AIMessage(content="Large language model"),         ])     }     | rag_prompt_custom     | llm     | StrOutputParser() ) rag_chain_with_source = RunnableParallel(     {"documents": retriever, "question": RunnablePassthrough()} ) | {     "documents": lambda input: [doc.metadata for doc in input["documents"]],     "answer": rag_chain_from_docs, }  rag_chain_with_source.invoke("What is Task Decomposition")

from operator import itemgetter
from langchain_core.runnables import RunnableParallel
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import VertexAI



In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnableParallel

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Chat History:
{chat_history}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def format_history(chat_history):
    return "\n\n".join(f'{chat.type} : {chat.content}\n' for chat in chat_history)



rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
        "chat_history": lambda input: format_history ([
            HumanMessage(content="What game was alice playing with the queen"),
            AIMessage(content="croquet"),
        ])
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke("Why was it difficult?")

{'documents': [{'source': 'alice_in_wonderland.txt',
   'author': 'Lewis',
   'book': 'Alice'},
  {'source': 'alice_in_wonderland.txt', 'author': 'Lewis', 'book': 'Alice'},
  {'source': 'Sorcerer_stone.txt', 'author': 'Rowling', 'book': 'Stone'},
  {'source': 'Sorcerer_stone.txt', 'author': 'Rowling', 'book': 'Stone'}],
 'answer': ' It was difficult because the flamingo kept twisting around and looking up at her with a puzzled expression, the hedgehog kept unrolling itself and crawling away, and there were always ridges or furrows in the way wherever she wanted to send the hedgehog. Thanks for asking!'}

In [ ]:
from langchain.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

' Il testo fornito non contiene informazioni sul luogo di lavoro di Harrison.'

In [ ]:
# https://python.langchain.com/docs/expression_language/cookbook/retrieval
# https://colab.research.google.com/drive/10kA1-cYf0oiHDKUqEHgA-5S0-5hAVo-W#scrollTo=ded41680-b749-4e2a-9daa-b1165d74783b

# stream
# https://python.langchain.com/docs/integrations/chat/google_vertex_ai_palm




In [ ]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)



DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | llm

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

' The provided text does not mention where Harrison works.'

In [ ]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

' The provided text does not mention where Harrison works.'

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

inputs = {"question": "where did harry live?"}
result = final_chain.invoke(inputs)
result

{'answer': ' Harry lived with the Dursleys, his aunt, uncle, and cousin, in a house on Privet Drive.',
 'docs': [Document(page_content='Hagrid was so huge that he parted the crowd easily; all Harry had to do\nwas keep close behind him. They passed book shops and music stores,\nhamburger restaurants and cinemas, but nowhere that looked as if it\ncould sell you a magic wand. This was just an ordinary street full of\nordinary people. Could there really be piles of wizard gold buried miles\nbeneath them? Were there really shops that sold spell books and\nbroomsticks? Might this not all be some huge joke that the Dursleys had\ncooked up? If Harry hadn\'t known that the Dursleys had no sense of\nhumor, he might have thought so; yet somehow, even though everything\nHagrid had told him so far was unbelievable, Harry couldn\'t help\ntrusting him.\n\n"This is it," said Hagrid, coming to a halt, "the Leaky Cauldron. It\'s a\nfamous place."', metadata={'source': 'Sorcerer_stone.txt', 'author': 'Ro

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory


_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)


# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history= itemgetter("chat_history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

inputs = {"question": "why was it difficult?" ,
          "chat_history" : [
            HumanMessage(content="What game was alice playing with the queen"),
            AIMessage(content="croquet"),
        ]
          }
result = final_chain.invoke(inputs)
result

{'answer': " C'était difficile parce que le hérisson se déroulait et s'éloignait, que les soldats doublés se levaient et s'éloignaient vers d'autres parties du terrain, et qu'il y avait généralement une crête ou un sillon sur le chemin où elle voulait envoyer le hérisson.",
 'docs': [Document(page_content="It sounded an excellent plan, no doubt, and very neatly and\nsimply arranged; the only difficulty was, that she had not the\nsmallest idea how to set about it; and while she was peering\nabout anxiously among the trees, a little sharp bark just over\nher head made her look up in a great hurry.\n\n  An enormous puppy was looking down at her with large round\neyes, and feebly stretching out one paw, trying to touch her.\n`Poor little thing!' said Alice, in a coaxing tone, and she tried\nhard to whistle to it; but she was terribly frightened all the\ntime at the thought that it might be hungry, in which case it\nwould be very likely to eat her up in spite of all her coaxing.", metadata=

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory


template = """Answer the question based only on the following context:
{context}
Chat History:
{chat_history}
Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)


# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history= itemgetter("chat_history"),
    question=itemgetter("question")
)
# Now we calculate the standalone question

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("question") | retriever,
    "question": lambda x: x["question"],
    "chat_history": lambda x: get_buffer_string(x["chat_history"]),
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
    "chat_history" : itemgetter("chat_history")
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory |  retrieved_documents | answer

inputs = {"question": "Can you describe him? Whom he usually interact with?" ,
          "chat_history" : [
            HumanMessage(content="Who taught defence and seceret arts"),
            AIMessage(content="Quirrell"),
        ]
          }
result = final_chain.invoke(inputs)
result

100%|██████████| 690/690 [00:00<00:00, 4022.80it/s]


{'answer': ' Quirrell is a professor at Hogwarts School of Witchcraft and Wizardry. He is described as a nervous and stuttering man with a turban on his head. He is the professor of Defence Against the Dark Arts and is known for his fear of everything. He often interacts with Harry Potter, as he is one of his students.',
 'docs': [Document(page_content='"Shut up," said Ron again. He was almost as tall as the twins already\nand his nose was still pink where his mother had rubbed it.\n\n"Hey, Mom, guess what? Guess who we just met on the train?"\n\nHarry leaned back quickly so they couldn\'t see him looking.\n\n"You know that black-haired boy who was near us in the station? Know who\nhe is?"\n\n"Who?"\n\n"Harry Potter!"\n\nHarry heard the little girl\'s voice.\n\n"Oh, Mom, can I go on the train and see him, Mom, eh please...."\n\n"You\'ve already seen him, Ginny, and the poor boy isn\'t something you\ngoggle at in a zoo. Is he really, Fred? How do you know?"\n\n"Asked him. Saw his scar. 

In [27]:
! pip install -q fastembed faiss-cpu unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 42.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=51573b1dad3f87c52cd2373183ecc0fc9cd134cfbb50e68b3c0b893b073da58a
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take int

In [12]:
from langchain.embeddings.fastembed import FastEmbedEmbeddings
from langchain.vectorstores import FAISS
embed_model = FastEmbedEmbeddings()

db = FAISS.from_documents(alice_docs,embed_model)

In [13]:
docs = db.similarity_search("alice queen play game")
print(docs[0].page_content)

Alice thought she might as well go back, and see how the game
was going on, as she heard the Queen's voice in the distance,
screaming with passion.  She had already heard her sentence three
of the players to be executed for having missed their turns, and
she did not like the look of things at all, as the game was in
such confusion that she never knew whether it was her turn or
not.  So she went in search of her hedgehog.

  The hedgehog was engaged in a fight with another hedgehog,
which seemed to Alice an excellent opportunity for croqueting one
of them with the other:  the only difficulty was, that her
flamingo was gone across to the other side of the garden, where
Alice could see it trying in a helpless sort of way to fly up
into a tree.


In [14]:
retriever=db.as_retriever()

In [21]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory

from langchain.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "what game were alice and queen playing and why was it difficult", "language": "english"})

' Alice and the Queen were playing a game of croquet. The game was difficult because the croquet ground was full of ridges and furrows, the balls were live hedgehogs, the mallets were live flamingos, and the soldiers had to double themselves up and stand on their hands and feet to make the arches.'

In [23]:
#https://python.langchain.com/docs/integrations/vectorstores/faiss
db.save_local("faiss_index")

new_db = FAISS.load_local("faiss_index", embed_model)

In [22]:
!ls -alrth

total 612K
drwxr-xr-x 1 root root 4.0K Dec 28 14:19 sample_data
drwxr-xr-x 1 root root 4.0K Jan  2 22:13 ..
-rw-r--r-- 1 root root 146K Jan  2 22:29 alice_in_wonderland.txt
-rw-r--r-- 1 root root 430K Jan  2 22:29 Sorcerer_stone.txt
drwxr-xr-x 7 root root 4.0K Jan  2 22:30 deeplk
drwxr-xr-x 1 root root 4.0K Jan  2 22:34 .config
drwx------ 5 root root 4.0K Jan  2 22:34 drive
drwxr-xr-x 1 root root 4.0K Jan  2 22:34 .


In [24]:
import requests

headers = {
    'Referer': 'https://www.google.com/',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

response = requests.get('https://www.cardinalhealth.com/sitemap.xml', headers=headers)

from bs4 import BeautifulSoup

#response = get_response('https://www.cardinalhealth.com/sitemap.xml')
soup = BeautifulSoup(response.content, "xml")
urls = [element.text for element in soup.find_all("loc")]

sites = urls
sites_filtered = [url for url in sites if '/reference/' not in url and '?hl' not in url]
sites_filtered[:5]

['https://www.cardinalhealth.com',
 'https://www.cardinalhealth.com/en.html',
 'https://www.cardinalhealth.com/en/about-us.html',
 'https://www.cardinalhealth.com/en/about-us/who-we-are.html',
 'https://www.cardinalhealth.com/en/about-us/who-we-are/fuse-by-cardinal-health.html']

In [25]:
diversitysites = [url for url in sites if 'diversity' in url]
len(diversitysites),diversitysites

(16,
 ['https://www.cardinalhealth.com/en/about-us/who-we-are/supplier-diversity.html',
  'https://www.cardinalhealth.com/en/about-us/who-we-are/supplier-diversity/customers.html',
  'https://www.cardinalhealth.com/en/about-us/who-we-are/supplier-diversity/suppliers.html',
  'https://www.cardinalhealth.com/en/about-us/our-people/diversity-and-inclusion.html',
  'https://www.cardinalhealth.com/en/about-us/our-people/diversity-and-inclusion/diversity-inclusion-steering-council.html',
  'https://www.cardinalhealth.com/en/about-us/our-people/diversity-and-inclusion/african-american-and-black-racial-equity-cabinet.html',
  'https://www.cardinalhealth.com/en/about-us/our-people/diversity-and-inclusion/employee-resource-groups.html',
  'https://www.cardinalhealth.com/en/about-us/our-people/diversity-and-inclusion/driving-innovation-through-dei.html',
  'https://www.cardinalhealth.com/en/about-us/our-people/diversity-and-inclusion/dei-by-the-numbers.html',
  'https://www.cardinalhealth.com/en/

In [28]:

from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#loader = UnstructuredURLLoader(urls=sites_filtered)
#documents = loader.load()

loader = UnstructuredURLLoader(urls=diversitysites[:15], continue_on_failure=False, headers=headers)
documents = loader.load()


#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 950,
    chunk_overlap  = 90,
    length_function = len,
)

documents = text_splitter.split_documents(documents)
print(len(documents))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


818


In [29]:
vectorstore = FAISS.from_documents(documents, embed_model)